# Building up on Question 2.1.1 , adding min,max,std,skew,mean and tsfresh features

In [1]:
# Core Libraries
import os
import glob
import itertools

# Data Handling
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Progress Bar
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import export_graphviz
from IPython.display import Image
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc

# Statistics
from scipy.stats import skew, kurtosis, randint

# Modeling
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    average_precision_score,
    recall_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn import linear_model

In [2]:

static_variables = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType', 'Weight']

static_variables.remove('ICUType')

static_variables_we_want = ['Age', 'Gender', 'Height', 'Weight']
all_variables = ['Weight', 'Age', 'TroponinI', 'DiasABP', 'MechVent', 'HCO3', 'Cholesterol', 'HCT', 'SaO2', 'WBC', 'SysABP', 'Urine', 'ICUType', 'Gender', 'ALP', 'Creatinine', 'K', 'AST', 'Glucose', 'RespRate', 'MAP', 'FiO2', 'BUN', 'Na', 'Bilirubin', 'TroponinT', 'PaCO2', 'Height', 'GCS', 'HR', 'pH', 'PaO2', 'Lactate', 'ALT', 'NISysABP', 'RecordID', 'Platelets', 'Temp', 'Mg', 'NIDiasABP', 'Albumin', 'NIMAP']
dyn_variables = [x for x in all_variables if x not in static_variables]
dyn_variables.remove('ICUType')
dyn_variables.append('Weight_VAR')
len(dyn_variables), len(static_variables_we_want)

initial_column_lists = static_variables_we_want + dyn_variables

In [3]:

# import parquet file 
df_a =pd.read_parquet('data/processed_raw_data_set-a_1.parquet', engine='pyarrow')
df_b =pd.read_parquet('data/processed_raw_data_set-b_1.parquet', engine='pyarrow')
df_c =pd.read_parquet('data/processed_raw_data_set-c_1.parquet', engine='pyarrow')

drop_ICUType = True 
if drop_ICUType:
    df_a = df_a.drop(columns=['ICUType'])
    df_b = df_b.drop(columns=['ICUType'])
    df_c = df_c.drop(columns=['ICUType'])


#  drop Time variable in df_a
if 'Time' in df_a.columns:
    df_a = df_a.drop(columns=['Time'])
    df_b = df_b.drop(columns=['Time'])
    df_c = df_c.drop(columns=['Time'])




## Computing features vectors of our patient

Instead of working on the table where the missing values had already been filled, i prefer working on the not filled table because otherwise filled values would be taken into the mean and might flatten patient with lots of missing values. Then I compute the mean of variables for eahc patient over the 49 timestamps. 

Then i compute the median on the resulting table to fill the missing values

In [4]:
len(static_variables_we_want) , len(dyn_variables), len(static_variables_we_want) + len(dyn_variables) 

(4, 37, 41)

In [5]:
# we define aggregation functions
agg_funcs = {col: ['mean','std','max','min','skew'] for col in dyn_variables}  

agg_funcs['RecordID'] = 'first'  # Keep RecordID
for stat_var in static_variables_we_want:
    if stat_var in df_a.columns:
        agg_funcs[stat_var] = 'first'  # Keep static variables

# Compute mean and std in one go
df_a_agg = df_a.groupby('RecordID').agg(agg_funcs)

df_a_agg.columns = ['_'.join(col).strip() for col in df_a_agg.columns.values]


In [6]:
# same for df_b 
agg_funcs = {col: ['mean','std','max','min','skew'] for col in dyn_variables}
agg_funcs['RecordID'] = 'first'  # Keep RecordID
for stat_var in static_variables_we_want:
    if stat_var in df_b.columns:
        agg_funcs[stat_var] = 'first'  # Keep static variables

# Compute mean and std in one go
df_b_agg = df_b.groupby('RecordID').agg(agg_funcs)

df_b_agg.columns = ['_'.join(col).strip() for col in df_b_agg.columns.values]


In [7]:
# same for df_c
agg_funcs = {col: ['mean','std','max','min','skew'] for col in dyn_variables}
agg_funcs['RecordID'] = 'first'  # Keep RecordID
for stat_var in static_variables_we_want:
    if stat_var in df_c.columns:
        agg_funcs[stat_var] = 'first'  # Keep static variables

# Compute mean and std in one go
df_c_agg = df_c.groupby('RecordID').agg(agg_funcs)
df_c_agg.columns = ['_'.join(col).strip() for col in df_c_agg.columns.values]

In [8]:
# compute median of df_a 

df_a_agg_median = df_a_agg.median()

# fill missing values with median
df_a_agg.fillna(df_a_agg_median, inplace=True)

df_b_agg.fillna(df_a_agg_median, inplace=True)
df_c_agg.fillna(df_a_agg_median, inplace=True)

In [9]:
# (df_a_agg.isnull().sum() != 0 ) print where true
# print columns with missing values
missing_values_a = df_a_agg.isnull().sum() != 0
#  print only where true
missing_values_a = missing_values_a[missing_values_a].index.tolist()
print("Missing values in df_a_agg:", missing_values_a)

Missing values in df_a_agg: ['Cholesterol_skew']


In [10]:
missing_values_b = df_b_agg.isnull().sum() != 0
#  print only where true
missing_values_b = missing_values_b[missing_values_b].index.tolist()
print("Missing values in df_b_agg:", missing_values_b)

Missing values in df_b_agg: ['Cholesterol_skew']


In [11]:
missing_values_c = df_c_agg.isnull().sum() != 0
#  print only where true
missing_values_c = missing_values_c[missing_values_c].index.tolist()
print("Missing values in df_c_agg:", missing_values_c)

Missing values in df_c_agg: ['Cholesterol_skew']


In [12]:
# drop Cholesterol_skew in df_a_agg, df_b_agg, df_c_agg
df_a_agg = df_a_agg.drop(columns=['Cholesterol_skew'])
df_b_agg = df_b_agg.drop(columns=['Cholesterol_skew'])
df_c_agg = df_c_agg.drop(columns=['Cholesterol_skew'])

In [13]:
len(df_a_agg['Weight_VAR_mean'].unique()), len(df_a_agg['Weight_VAR_std'].unique()) 

(2110, 1471)

In [14]:
assert df_a_agg.isnull().sum().sum() == 0
assert df_b_agg.isnull().sum().sum() == 0
assert df_c_agg.isnull().sum().sum() == 0

# Tsfresh

In [15]:
# open set-a_no_nan.parquet
df_a_no_nan = pd.read_parquet('data/set-a_no_nan.parquet', engine='pyarrow')
df_b_no_nan = pd.read_parquet('data/set-b_no_nan.parquet', engine='pyarrow')
df_c_no_nan = pd.read_parquet('data/set-c_no_nan.parquet', engine='pyarrow')


In [16]:
df_a_no_nan.columns

Index(['RecordID', 'Time', 'Age', 'Gender', 'Height', 'Weight', 'Albumin',
       'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol', 'Creatinine',
       'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT', 'HR', 'K',
       'Lactate', 'Mg', 'MAP', 'MechVent', 'Na', 'NIDiasABP', 'NIMAP',
       'NISysABP', 'PaCO2', 'PaO2', 'pH', 'Platelets', 'RespRate', 'SaO2',
       'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC',
       'Weight_VAR'],
      dtype='object', name='Parameter')

In [17]:
from tsfresh import extract_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table


In [18]:
outcomes = pd.read_csv('data/outcomes.csv')

outcomes_a = outcomes.loc[outcomes['RecordID'].isin(df_a_no_nan['RecordID'])]
# outcomes_a set index to RecordID
outcomes_a = outcomes_a.set_index('RecordID')
outcomes_a 

,In-hospital_death
RecordID,
132539,0
132540,0
132541,0
132543,0
132545,0
...,...
142665,0
142667,0
142670,0


## To compute Tsfresh features on training set 

In [ ]:
from tsfresh import extract_features
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.utilities.dataframe_functions import impute
import pandas as pd
from functools import reduce
from pandas import DataFrame
from tqdm import tqdm

we_compute_training_features = False

if we_compute_training_features:
    # 1. Load data
    df_a_no_nan = pd.read_parquet('data/set-a_no_nan.parquet', engine='pyarrow')
    outcomes = pd.read_csv('data/outcomes.csv')
    outcomes_a = outcomes.loc[outcomes['RecordID'].isin(df_a_no_nan['RecordID'])]
    outcomes_a = outcomes_a.set_index('RecordID')

    # 2. Define your dynamic variables (update this list as needed)
    # cf above

    # 3. Empty list to collect features for all variables
    all_feature_sets = []

    # 4. Loop through each variable
    for i, var in tqdm(enumerate(dyn_variables), desc="Processing variables"):
        print(f"Processing {i}, {var}...")
        if (i <26):
            continue
        # if (i==26):
        #     break
        
        df_long = df_a_no_nan[['RecordID', 'Time', var]].copy()
        df_long = df_long.rename(columns={var: 'value'})  # tsfresh expects 'value'
        
        # extract tsfresh features
        features = extract_features(df_long, column_id='RecordID', column_sort='Time', n_jobs=8)
        
        # drop features with NaNs
        features = features.dropna(axis=1, how='any')
        if features.empty:
            continue  # skip if nothing left
        
        # align with labels
        labels = outcomes_a.loc[features.index]['In-hospital_death']
        
        # calculate relevance
        relevance_table = calculate_relevance_table(features, labels, ml_task='classification')
        top_features = relevance_table[relevance_table.relevant].sort_values("p_value")["feature"][:5]
        
        # reduce to top 5 features and rename
        selected = features[top_features]
        selected.columns = [f"{var}__{col}" for col in selected.columns]
        
        # collect
        all_feature_sets.append(selected)

    # 5. Combine all

    tsfresh_final_features = reduce(lambda left, right: left.join(right, how='outer'), all_feature_sets)

    # 6. Final cleanup
    tsfresh_final_features = tsfresh_final_features.fillna(tsfresh_final_features.median())

    print("Final shape:", tsfresh_final_features.shape)
    tsfresh_final_features.head()

    # save tsfresh_final_features
    tsfresh_final_features.to_parquet('data/tsfresh_final_features_26.parquet', engine='pyarrow', index=True)


## Retrieving the same tsfresh features as in the traning set for the test set 

In [21]:
from collections import defaultdict


In [22]:
# tsfresh_final_features_10 = pd.read_parquet('data/tsfresh_final_features_10.parquet', engine='pyarrow') #CHANGE
# tsfresh_final_features_10.columns
# tsfresh_final_features_11_25 = pd.read_parquet('data/tsfresh_final_features_11_25.parquet', engine='pyarrow')
# tsfresh_final_features_11_25.columns
tsfresh_final_features_26 = pd.read_parquet('data/tsfresh_final_features_26.parquet', engine='pyarrow')
tsfresh_final_features_26.columns


Index(['PaO2__value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
       'PaO2__value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',
       'PaO2__value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',
       'PaO2__value__count_above_mean',
       'PaO2__value__approximate_entropy__m_2__r_0.7',
       'Lactate__value__sum_of_reoccurring_values',
       'Lactate__value__approximate_entropy__m_2__r_0.3',
       'Lactate__value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
       'Lactate__value__approximate_entropy__m_2__r_0.1',
       'Lactate__value__sample_entropy',
       'ALT__value__ratio_value_number_to_time_series_length',
       'ALT__value__approximate_entropy__m_2__r_0.7',
       'ALT__value__spkt_welch_density__coeff_2',
       'ALT__value__lempel_ziv_complexity__bins_100',
       'ALT__value__fft_coefficient__attr_"abs"__coeff_24',
       'NISysABP__value__agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',


In [23]:
def group_by_first_double_underscore(feature_list):
    grouped = defaultdict(list)
    for item in feature_list:
        if '__' in item:
            key, rest = item.split('__', 1)  # split only at the first occurrence
            grouped[key].append(rest)
    return dict(grouped)


In [24]:
dico_selected = group_by_first_double_underscore(tsfresh_final_features_26.columns.tolist()) #CHANGE
dico_selected

{'PaO2': ['value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0',
  'value__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0',
  'value__change_quantiles__f_agg_"var"__isabs_True__qh_0.6__ql_0.0',
  'value__count_above_mean',
  'value__approximate_entropy__m_2__r_0.7'],
 'Lactate': ['value__sum_of_reoccurring_values',
  'value__approximate_entropy__m_2__r_0.3',
  'value__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2',
  'value__approximate_entropy__m_2__r_0.1',
  'value__sample_entropy'],
 'ALT': ['value__ratio_value_number_to_time_series_length',
  'value__approximate_entropy__m_2__r_0.7',
  'value__spkt_welch_density__coeff_2',
  'value__lempel_ziv_complexity__bins_100',
  'value__fft_coefficient__attr_"abs"__coeff_24'],
 'NISysABP': ['value__agg_linear_trend__attr_"rvalue"__chunk_len_5__f_agg_"var"',
  'value__agg_linear_trend__attr_"rvalue"__chunk_len_10__f_agg_"var"',
  'value__agg_linear_trend__attr_"slope"__chunk_len_10__f_agg_"var"',
  'value

In [25]:
from tsfresh import extract_features
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.utilities.dataframe_functions import impute
import pandas as pd
from functools import reduce
from pandas import DataFrame
from tqdm import tqdm


# 1. Load data
df_c_no_nan = pd.read_parquet('data/set-c_no_nan.parquet', engine='pyarrow')
outcomes = pd.read_csv('data/outcomes.csv')

In [26]:
# df_c_no_nan = df_c_no_nan.iloc[:490, :]

In [27]:

# 2. Define your dynamic variables (update this list as needed)
# cf above

# 3. Empty list to collect features for all variables
all_feature_sets = []

# 4. Loop through each variable
for i, var in tqdm(enumerate(dyn_variables), desc="Processing variables"):
    print(f"Processing {i}, {var}...")
    if var not in dico_selected.keys():
        print(var)
        print('not there')
        continue

    # if (i <10): 
    #     continue
    if (i <26): 
        continue
    # if (i==26): # change !
    #     break
    
    df_long = df_c_no_nan[['RecordID', 'Time', var]].copy()
    df_long = df_long.rename(columns={var: 'value'})  # tsfresh expects 'value'
    
    # extract tsfresh features
    features = extract_features(df_long, column_id='RecordID', column_sort='Time', n_jobs=8)


    
    # drop features with NaNs
    # features = features.dropna(axis=1, how='any')
    if features.empty:
        continue  # skip if nothing left
    
    top_features = dico_selected[var]
    selected = features[top_features]
    selected.columns = [f"{var}__{col}" for col in selected.columns]


    
    # collect
    all_feature_sets.append(selected)

# 5. Combine all

tsfresh_final_features = reduce(lambda left, right: left.join(right, how='outer'), all_feature_sets)

# tsfresh_final_features = tsfresh_final_features['TroponinI__value__fourier_entropy__bins_2']

print("Final shape:", tsfresh_final_features.shape)
tsfresh_final_features.head()


Processing variables: 0it [00:00, ?it/s]

Processing 0, TroponinI...
TroponinI
not there
Processing 1, DiasABP...
DiasABP
not there
Processing 2, MechVent...
MechVent
not there
Processing 3, HCO3...
HCO3
not there
Processing 4, Cholesterol...
Cholesterol
not there
Processing 5, HCT...
HCT
not there
Processing 6, SaO2...
SaO2
not there
Processing 7, WBC...
WBC
not there
Processing 8, SysABP...
SysABP
not there
Processing 9, Urine...
Urine
not there
Processing 10, ALP...
ALP
not there
Processing 11, Creatinine...
Creatinine
not there
Processing 12, K...
K
not there
Processing 13, AST...
AST
not there
Processing 14, Glucose...
Glucose
not there
Processing 15, RespRate...
RespRate
not there
Processing 16, MAP...
MAP
not there
Processing 17, FiO2...
FiO2
not there
Processing 18, BUN...
BUN
not there
Processing 19, Na...
Na
not there
Processing 20, Bilirubin...
Bilirubin
not there
Processing 21, TroponinT...
TroponinT
not there
Processing 22, PaCO2...
PaCO2
not there
Processing 23, GCS...
GCS
not there
Processing 24, HR...
HR
not th


Feature Extraction: 100%|██████████| 40/40 [03:13<00:00,  4.84s/it]
Processing variables: 27it [03:17,  7.31s/it]

Processing 27, Lactate...



Feature Extraction: 100%|██████████| 40/40 [03:00<00:00,  4.50s/it]
Processing variables: 28it [06:21, 16.21s/it]

...


Feature Extraction: 100%|██████████| 40/40 [03:04<00:00,  4.61s/it]
Processing variables: 36it [32:00, 142.43s/it]

Processing 36, Weight_VAR...



Feature Extraction: 100%|██████████| 40/40 [02:58<00:00,  4.47s/it]
Processing variables: 37it [35:02, 56.83s/it] 

Final shape: (4000, 55)


,"PaO2__value__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.0","PaO2__value__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.0","PaO2__value__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0",PaO2__value__count_above_mean,PaO2__value__approximate_entropy__m_2__r_0.7,Lactate__value__sum_of_reoccurring_values,Lactate__value__approximate_entropy__m_2__r_0.3,"Lactate__value__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2",Lactate__value__approximate_entropy__m_2__r_0.1,Lactate__value__sample_entropy,...,"NIMAP__value__agg_linear_trend__attr_""rvalue""__chunk_len_5__f_agg_""var""","NIMAP__value__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""var""","NIMAP__value__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","NIMAP__value__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""min""","NIMAP__value__agg_linear_trend__attr_""rvalue""__chunk_len_10__f_agg_""max""",Weight_VAR__value__energy_ratio_by_chunks__num_segments_10__segment_focus_2,"Weight_VAR__value__change_quantiles__f_agg_""mean""__isabs_False__qh_0.6__ql_0.2","Weight_VAR__value__change_quantiles__f_agg_""var""__isabs_False__qh_0.6__ql_0.2","Weight_VAR__value__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.2","Weight_VAR__value__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2"
152871.0,2.000000,33.683594,29.687500,23.0,0.474895,4.2,0.140442,0.019048,0.140442,0.099967,...,0.000000,-0.707107,75.330,75.330000,0.000000,0.086122,0.262162,2.474244,2.474244,0.262162
152873.0,1.228571,12.404898,11.033469,27.0,0.304590,10.8,0.421972,0.016667,0.421972,0.351761,...,0.359495,0.511459,70.266,70.761636,0.047594,0.086419,0.220455,2.089809,2.089809,0.220455
152875.0,0.000000,0.000000,0.000000,0.0,0.000000,2.0,0.000000,0.000000,0.000000,0.042560,...,-0.556346,-0.624538,70.930,74.005091,-0.838189,0.107422,-0.095238,0.371882,0.371882,0.095238
152878.0,0.000000,0.000000,0.000000,0.0,0.000000,2.0,0.000000,0.000000,0.000000,0.042560,...,0.542190,0.629700,74.398,77.465182,0.428084,0.102041,0.000000,0.000000,0.000000,0.000000
152882.0,1.393939,21.702479,20.056933,25.0,0.262517,4.3,0.083417,0.006250,0.083417,0.075476,...,-0.522233,-0.707107,67.530,70.839091,0.000000,0.102041,0.000000,0.000000,0.000000,0.000000


In [28]:
# save tsfresh_final_features CHANGE
tsfresh_final_features.to_parquet('data/tsfresh_final_features_C_10.parquet', engine='pyarrow', index=True)
# tsfresh_final_features.to_parquet('data/tsfresh_final_features_C_11_25.parquet', engine='pyarrow', index=True)
# tsfresh_final_features.to_parquet('data/tsfresh_final_features_C_26.parquet', engine='pyarrow', index=True)